In [1]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.model_selection import train_test_split

In [3]:

# Define the model and tokenizer names for Indic BERT for Hindi
model_name = "ai4bharat/indic-bert"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)




In [4]:
!pip3 install pandas_ods_reader
from pandas_ods_reader import read_ods
newdf=read_ods("/content/drive/MyDrive/iisc/reverse_dictionary/hi_3500.ods",headers=False)
newdf.columns.values[0]='text'
newdf.columns.values[1]='label'
newdf.head()
newdf.label=pd.Categorical(newdf.label)
newdf['label']=newdf['label'].cat.codes
newdf = newdf.dropna()
newdf.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49002 sha256=b1a35b4f2138dcd530df440f4675903ef57a7f7d92311cc6b9b84edb2ecc777f
  Stored in directory: /root/.cache/pip/wheels/50/09/62/ea2c44e6ffdd067d6c8d3f557af0734a195bd252d6901f23ca
Successfully built ezodf


,text,label
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,0
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,0
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,0
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,0
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,0


In [5]:
input=newdf['text']
Y = pd.get_dummies(newdf['label']).values
print(Y)
onehot=Y

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [7]:
text = input[0]
print(text)

लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान जाएगा तू ले जाकर दिल्ली इसे दिखा ला दोस्त


In [8]:
max_sentence_length = 511

# List to store average embeddings
avg_embeddings_list = []

# Process each sentence in the DataFrame
for sentence in input:
    # Trim sentence length
    trimmed_sentence = sentence[:max_sentence_length]

    # Tokenize the input text and move to GPU
    inputs = tokenizer(trimmed_sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=max_sentence_length)
    inputs.to(device)

    # Forward pass to obtain embeddings
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    # Calculate the average embedding for the sentence
    avg_embedding = torch.mean(embeddings, dim=1)  # Average along the token dimension
    avg_embeddings_list.append(avg_embedding)

# Stack the average embeddings for all sentences
avg_embeddings_matrix = torch.cat(avg_embeddings_list, dim=0).cpu().numpy()

# Print the shape of the average embeddings matrix
print(avg_embeddings_matrix.shape)

(9077, 768)


In [9]:
path_new="/content/drive/MyDrive/iisc/reverse_dictionary/embedding_matrix_indicbert.npy"
np.save(path_new, avg_embeddings_matrix)
embedding_matrix=np.load(path_new)

In [10]:
x_train,x_test,y_train,y_test=train_test_split(embedding_matrix,Y,train_size=0.7,random_state=42)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Flatten,LSTM,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [12]:
def create_dan_model():
    model_dan = Sequential()
    model_dan.add(Dense(300, activation='linear', input_dim=768))
    model_dan.add(Dense(100, activation='elu'))
    model_dan.add(Dense(100, activation='elu'))
    model_dan.add(Dense(100, activation='elu'))
    model_dan.add(Dense(3, activation='softmax'))
    return model_dan

# Create the model
model_dan = create_dan_model()


In [14]:
model_dan.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model_dan.summary()
model_dan.fit(x_train, y_train, epochs=200, batch_size=64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               230700    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 3)                 303       
                                                                 
Total params: 281,303
Trainable params: 281,303
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
100/100 [=============================

In [15]:
loss, accuracy = model_dan.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

86/86 [==============================] - 1s 4ms/step - loss: 1.4478 - accuracy: 0.7775
Test loss: 1.4478, Test accuracy: 0.7775


In [16]:
from tabulate import tabulate
from PIL import Image, ImageDraw, ImageFont

# Data
data = [
    ["BERT-base-multilingual-uncased", "Test Accuracy: 78.28%"],
    ["Indic-bert by AI Bharat (768 dimensions)", "Test Accuracy: 77.77%"],
    ["monsoon-nlp/hindi-tpu-electra", "Test Accuracy: 76.37%"],
    ["monsoon-nlp/hindi-bert (256 dimensions)", "Test Accuracy: 67.84%"],
    ["FastText (300 dimensions)", "Test Accuracy: 64.54%"]
]

# Convert data to tabulated format
table = tabulate(data, headers=["Model", "Accuracy"], tablefmt="plain")

# Create an image with the table
image = Image.new("RGB", (800, 200), color="white")
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

# Draw the table text
draw.text((10, 10), table, font=font, fill="black")

# Save the image
image.save("accuracy_table.png")